# Problem set 6: Solving the Solow model

In [1]:
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm

# Tasks

## Solving matrix equations I

In [2]:
np.random.seed(1900)
n = 5
A = np.random.uniform(size=(n,n))
b = np.random.uniform(size=n)
c = np.random.uniform(size=n)
d = np.random.uniform(size=n)

**Question A:** Find the determinant of $[A \cdot A]^{-1}$

In [7]:
# write your code here
linalg.det(linalg.inv(A@A))

13132.548734461372

**Answer:** see A1.py

**Question B:** Solve the following equation systems directly using **scipy**.

$$
\begin{aligned}
Ax &= b \\
Ax &= c \\
Ax &= d 
\end{aligned}
$$

In [16]:
sol_b = linalg.inv(A)@b
sol_c = linalg.inv(A)@c
sol_d = linalg.inv(A)@d

np.allclose(A.dot(sol_d), d)

True

**Answer:** A2.py

**Question C:** Solve the same equation systems as above using `linalg.lu_factor()` and `linalg.lu_solve()`. What is the benefit of this approach?

In [22]:
LU, piv = linalg.lu_factor(A)
x = linalg.lu_solve((LU, piv), b)
print(x)
print(sol_b)

# This approach is faster

[-15.33189031 -24.00998148  40.02675108  15.24193293   4.89008792]
[-15.33189031 -24.00998148  40.02675108  15.24193293   4.89008792]


**Answer:** A3.py

## Solving matrix equations II

In [34]:
F = np.array([[2.0, 1.0, -1.0], [-3.0, -1.0, 2], [-2.0, 1.0, 2.0]])
e = np.array([8.0, -11.0, -3.0])

A = np.column_stack((F, e))
print(A)

[[  2.   1.  -1.   8.]
 [ -3.  -1.   2. -11.]
 [ -2.   1.   2.  -3.]]
[[ 2.  1. -1.]
 [-3. -1.  2.]
 [-2.  1.  2.]]


**Question:** Use the function `gauss_jordan()` in the `numecon_linalg` module located in this folder to solve

$$
Fx = e
$$

In [23]:
from numecon_linalg import *

In [52]:
gauss_jordan(A)
x = A[:, -1]

xs = linalg.solve(F, e)

print(x)
print(xs)

[ 2.  3. -1.]
[ 2.  3. -1.]


**Answer:** see A4.py

## Symbolic

**Question A:** Find

$$
\lim_{x \rightarrow 0} \frac{\sin(x)}{x}
$$

and

$$
\frac{\partial\sin(2x)}{\partial x} 
$$

In [55]:
# write your code here
x = sm.symbols("x")
sm.limit(sm.sin(x)/x, x, 0)

1

In [67]:
sm.Derivative(sm.sin(2*x), x, evaluate=True)

2*cos(2*x)

**Answer:** A5.py

**Question B:** Solve the equation

$$ 
\frac{\sin(x)}{x} = 0
$$

In [70]:
# write your code here
eq = sm.sin(x)/x
sol = sm.solve(sm.Eq(eq, 0), x)
print(sol[0])

pi


**Answer:** A6.py

# Problem: Solve the Solow model

## Introduction

Consider the **standard Solow-model** where:

1. $K_t$ is capital2
2. $L_t$ is labor (growing with a constant rate of $n$)
3. $A_t$ is technology (growing with a constant rate of $g$)
4. $Y_t = F(K_t,A_tL_t)$ is GDP

**Saving** is a constant fraction of GDP

$$ 
S_t = sY_t,\,s\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=S_{t}+(1-\delta)K_{t}=sF(K_{t},A_{t}L_{t})+(1-\delta)K_{t}, \delta \in (0,1)
$$

The **production function** has **constant-return to scale** such that

$$
\frac{Y_{t}}{A_{t}L_{t}}=\frac{F(K_{t},A_{t}L_{t})}{A_{t}L_{t}}=F(\tilde{k}_{t},1)\equiv f(\tilde{k}_{t})
$$

where $\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$ is the technology adjusted capital-labor ratio.

The **transition equation** then becomes

$$
\tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}_{t})+(1-\delta)\tilde{k}_{t}]
$$

If the **production function** is **Cobb-Douglas** then

$$
F(K_{t},A_{t}L_{t})=K_{t}^{\alpha}(A_{t}L_{t})^{1-\alpha}\Rightarrow f(\tilde{k}_{t})=\tilde{k}_{t}^{\alpha}
$$

If it is **CES** (with $\beta < 1, \beta \neq 0$) then

$$
F(K_{t},A_{t}L_{t})=(\alpha K_{t}^{\beta}+(1-\alpha)(A_{t}L_{t})^{\beta})^{\frac{1}{\beta}}\Rightarrow f(\tilde{k}_{t})=(\alpha\tilde{k}_{t}^{\beta}+(1-\alpha))^{\frac{1}{\beta}}
$$

## Steady state

Assume the production function is **Cobb-Douglas**.

**Question A:** Use **sympy** to find an analytical expression for the steady state, i.e. solve

$$
\tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}]
$$

In [127]:
k = sm.symbols('k')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')

In [128]:
# write your code here
objective = sm.Eq((s*k**alpha + (1-delta)*k)/((1 + n)*(1 + g)), k)
k_star = sm.solve(objective, k)
k_star[0]

((delta + g*n + g + n)/s)**(1/(alpha - 1))

**Answer:** see A7.py

**Question B:** Turn you solution into a Python function called as `ss_func(s,g,n,delta,alpha)`. 

In [135]:
ss_func = sm.lambdify((s,g,n,delta,alpha),k_star)

# Evaluate function
ss_func(0.2,0.02,0.01,0.1,1/3)

[1.903831539231319]

**Answer:** A8.py

**Question C**: Find the steady state numerically using root-finding with `optimize.root_scalar`.

In [136]:
s = 0.2
g = 0.02
n = 0.01
delta = 0.1
alpha = 1/3


def func(k):
    return (s*k**alpha + (1-delta)*k)/((1 + n)*(1 + g))-k

sol = optimize.root_scalar(func, bracket=[1e-8, 100], method='brentq')

In [137]:
sol.root

1.9038315392313188

**Answer:** A9.py

**Question D:** Now assume the production function is CES. Find the steady state for $k$ for the various values of $\beta$ shown below.

In [152]:
s = 0.2
g = 0.02
n = 0.01
delta = 0.1
alpha = 1/3
betas = [-0.5,-0.25,-0.1,-0.05, 1e-8,0.05,0.1,0.25,0.5]

In [154]:
# Evaluate function
for beta in betas:
    def func(k):
        ces = (alpha*k**beta + (1-alpha))**(1/beta)
        return (s*ces + (1-delta)*k)/((1 + n)*(1 + g))-k

    sol = optimize.root_scalar(func, bracket=[1e-8, 100], method='brentq')
    sol.root
    print(f'for beta = {beta}, then k_star = {sol.root:.4f}')


for beta = -0.5, then k_star = 1.8471
for beta = -0.25, then k_star = 1.8734
for beta = -0.1, then k_star = 1.8911
for beta = -0.05, then k_star = 1.8974
for beta = 1e-08, then k_star = 1.9038
for beta = 0.05, then k_star = 1.9105
for beta = 0.1, then k_star = 1.9174
for beta = 0.25, then k_star = 1.9396
for beta = 0.5, then k_star = 1.9822


**Answer:** A10.py   